# Target Encoding

In [1]:
import h2o
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

h2o.__version__

'3.36.0.4'

In [2]:
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.9" 2020-10-20 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.9+7-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.9+7-LTS, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpn_b_o__h
  JVM stdout: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpn_b_o__h/h2o_YShimada_MBP16_started_from_python.out
  JVM stderr: /var/folders/50/glkvvhjx03g22fwy2r0tgylr0000gn/T/tmpn_b_o__h/h2o_YShimada_MBP16_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Tokyo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,2 months and 26 days
H2O_cluster_name:,H2O_from_python_YShimada_MBP16_3vlam4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


データに関して: [H2O.ai Blog](https://h2o.ai/blog/driverless-ai-can-help-you-choose-what-you-consume-next/)

In [4]:
# 学習データ
fm_train = h2o.import_file('https://raw.githubusercontent.com/vopani/Recommendation-Datasets/main/movielens20m/movielens_train_sample.csv',
                          col_types={'movieId':'enum', 'userId':'enum'})
fm_train = fm_train[['movieId', 'userId', 'rating']]
fm_train.shape

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


(1000000, 3)

In [5]:
# テストデータ
fm_test = h2o.import_file('https://raw.githubusercontent.com/vopani/Recommendation-Datasets/main/movielens20m/movielens_test_sample.csv',
                                                    col_types={'movieId':'enum', 'userId':'enum'})
fm_test = fm_test[['movieId', 'userId', 'rating']]
fm_test.shape

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


(100000, 3)

In [6]:
fm_train.head(5)

movieId,userId,rating
1,91,4
1,221,4
1,238,4
1,280,4
1,367,3


In [7]:
y_col = 'rating'
X_cols = ['movieId', 'userId']

In [8]:
# 特徴量（カテゴリカル変数）の水準数 - 学習データ
pd.DataFrame({'Cols':fm_train[X_cols].columns, 'Type':fm_train[X_cols].types.values(), 'N_Levels':fm_train[X_cols].nlevels()})

,Cols,Type,N_Levels
0,movieId,enum,7972
1,userId,enum,7759


In [9]:
# 特徴量（カテゴリカル変数）の水準数 - テストデータ
pd.DataFrame({'Cols':fm_test[X_cols].columns, 'Type':fm_test[X_cols].types.values(), 'N_Levels':fm_test[X_cols].nlevels()})

,Cols,Type,N_Levels
0,movieId,enum,5919
1,userId,enum,2104


テストデータにあって、学習データに無い水準は削除しておく

In [10]:
# テストデータにあって、学習データに無い水準
ng_list_movieId = list(set(fm_test['movieId'].categories()) - set(fm_train['movieId'].categories()))
print(len(ng_list_movieId))
ng_list_userId = list(set(fm_test['userId'].categories()) - set(fm_train['userId'].categories()))
print(len(ng_list_userId))

118
513


In [11]:
fm_test = fm_test[~fm_test['movieId'].isin(ng_list_movieId)]
fm_test = fm_test[~fm_test['userId'].isin(ng_list_userId)]
fm_test.shape

(34333, 3)

## ベースラインモデル
カテゴリカルデータをそのままで利用

categorical_encodingに関して： [ドキュメント](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/categorical_encoding.html)

In [12]:
gbm_baseline = H2OGradientBoostingEstimator(categorical_encoding='auto')
%time gbm_baseline.train(x=X_cols, y=y_col, training_frame=fm_train, validation_frame=fm_test)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
CPU times: user 161 ms, sys: 38 ms, total: 199 ms
Wall time: 4.65 s
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1656200744705_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,63815.0,5.0,5.0,5.0,31.0,32.0,31.94




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.8575576566887294
RMSE: 0.9260440900349882
MAE: 0.7184197620178461
RMSLE: 0.2613434733252129
Mean Residual Deviance: 0.8575576566887294

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 0.7931498439576165
RMSE: 0.8905896046763719
MAE: 0.685720138393942
RMSLE: 0.25430354132122523
Mean Residual Deviance: 0.7931498439576165

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-06-26 08:46:08,0.038 sec,0.0,1.041483,0.818244,1.084688,1.003806,0.782945,1.007626
1,,2022-06-26 08:46:09,0.955 sec,1.0,1.035621,0.813121,1.072511,0.998687,0.778402,0.997376
2,,2022-06-26 08:46:09,1.303 sec,2.0,1.028956,0.807582,1.058751,0.992554,0.773344,0.985163
3,,2022-06-26 08:46:09,1.415 sec,3.0,1.023458,0.803362,1.047467,0.987637,0.769515,0.975426
4,,2022-06-26 08:46:09,1.501 sec,4.0,1.017987,0.798851,1.036298,0.982710,0.765424,0.965718
5,,2022-06-26 08:46:10,1.567 sec,5.0,1.014830,0.796395,1.029880,0.980004,0.763403,0.960409
6,,2022-06-26 08:46:10,1.630 sec,6.0,1.012157,0.794134,1.024462,0.977749,0.761588,0.955994
7,,2022-06-26 08:46:10,1.691 sec,7.0,1.008906,0.791165,1.017891,0.974624,0.758955,0.949893
8,,2022-06-26 08:46:10,1.753 sec,8.0,1.004970,0.787382,1.009965,0.970767,0.755486,0.942388
9,,2022-06-26 08:46:10,1.815 sec,9.0,1.002657,0.785182,1.005321,0.968530,0.753448,0.938050



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,movieId,917004.06250,1.000000,0.767097
1,userId,278416.46875,0.303615,0.232903


## Target Encodingを実施したモデル

[ドキュメント](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html)  
[API Reference](https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html#h2otargetencoderestimator)

In [13]:
# Target Encoding用のfoldカラム
fold_column = "kfold_column"
fm_train[fold_column] = fm_train.kfold_column(n_folds=5, seed=123)

In [14]:
fm_train.head(5)

movieId,userId,rating,kfold_column
1,91,4,1
1,221,4,1
1,238,4,1
1,280,4,2
1,367,3,1


In [15]:
# Target Mean Encoding
te = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",    # k foldを利用したTarget Encoding
                                       blending=True,        # 結果（水準別ターゲット変数の平均）に水準内のデータ数で重みつけを実施するかどうか
                                       inflection_point=10,         # blending=Trueの場合のパラメータ
                                       smoothing=20,            # blending=Trueの場合のパラメータ
                                       noise=0.01,            # Over fitを避けるためノイズの付与
                                       seed=123)
te

No model trained yet


In [16]:
te.train(x=X_cols,
              y=y_col,
              training_frame=fm_train)

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%
Model Details
H2OTargetEncoderEstimator :  TargetEncoder
Model Key:  TargetEncoder_model_python_1656200744705_2


Target Encoder model summary: Summary for target encoder model


,original_names,encoded_column_names
0,movieId,movieId_te
1,userId,userId_te


In [17]:
fm_train_te = te.transform(frame=fm_train, as_training=True)
print(fm_train_te.shape)
fm_test_te = te.transform(frame=fm_test)
print(fm_test_te.shape)

(1000000, 6)
(34333, 5)


In [18]:
fm_train_te.head(5)

movieId_te,userId_te,movieId,userId,kfold_column,rating
3.75589,3.50222,1,91,1,4
3.75526,3.15082,1,221,1,4
3.75843,3.46522,1,238,1,4
3.76599,4.04012,1,280,2,4
3.75634,2.79134,1,367,1,3


In [19]:
fm_test_te.head(5)

movieId_te,userId_te,movieId,userId,rating
3.74831,3.74033,1,5453,4
3.74768,3.84137,1,6141,3.5
3.75085,3.39111,1,9796,3
3.7625,4.18745,1,16959,4.5
3.74876,3.27797,1,25101,5


In [20]:
gbm_te = H2OGradientBoostingEstimator(categorical_encoding='auto')
%time gbm_te.train(x=['movieId_te', 'userId_te'], y=y_col, training_frame=fm_train_te, validation_frame=fm_test_te)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
CPU times: user 165 ms, sys: 39 ms, total: 204 ms
Wall time: 3.27 s
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1656200744705_3


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,22736.0,5.0,5.0,5.0,24.0,32.0,31.52




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.6771904114589722
RMSE: 0.822915798523137
MAE: 0.6245905835832357
RMSLE: 0.23147502622315805
Mean Residual Deviance: 0.6771904114589722

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 0.647307471280479
RMSE: 0.8045542065519756
MAE: 0.601536669940775
RMSLE: 0.2280791208098727
Mean Residual Deviance: 0.647307471280479

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-06-26 08:46:15,0.002 sec,0.0,1.041483,0.818244,1.084688,1.003806,0.782945,1.007626
1,,2022-06-26 08:46:15,0.130 sec,1.0,1.005587,0.787218,1.011206,0.967041,0.750841,0.935168
2,,2022-06-26 08:46:15,0.208 sec,2.0,0.975251,0.761437,0.951115,0.936207,0.724511,0.876484
3,,2022-06-26 08:46:15,0.287 sec,3.0,0.949779,0.740467,0.902080,0.910550,0.703438,0.829101
4,,2022-06-26 08:46:15,0.369 sec,4.0,0.928339,0.722807,0.861813,0.889255,0.686079,0.790775
5,,2022-06-26 08:46:15,0.451 sec,5.0,0.910435,0.707645,0.828893,0.872116,0.671559,0.760587
6,,2022-06-26 08:46:16,0.496 sec,6.0,0.895488,0.694762,0.801898,0.857783,0.658836,0.735792
7,,2022-06-26 08:46:16,0.551 sec,7.0,0.883087,0.683664,0.779842,0.846139,0.648004,0.715951
8,,2022-06-26 08:46:16,0.634 sec,8.0,0.872765,0.674167,0.761719,0.836536,0.638523,0.699792
9,,2022-06-26 08:46:16,0.686 sec,9.0,0.864185,0.666186,0.746816,0.828881,0.630915,0.687043



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,movieId_te,1354776.25,1.000000,0.63168
1,userId_te,789944.25,0.583081,0.36832


## 結果比較

In [21]:
print('----- ベースラインモデル -----')
print('Train R2: {}, Train RMSE{}'.format(gbm_baseline.r2(train=True), gbm_baseline.rmse(train=True)))
print('Test R2: {}, Test RMSE{}'.format(gbm_baseline.r2(valid=True), gbm_baseline.rmse(valid=True)))

print('----- Target Encodingを実施したモデル -----')
print('Train R2: {}, Train RMSE{}'.format(gbm_te.r2(train=True), gbm_te.rmse(train=True)))
print('Test R2: {}, Test RMSE{}'.format(gbm_te.r2(valid=True), gbm_te.rmse(valid=True)))

----- ベースラインモデル -----
Train R2: 0.20939663875850412, Train RMSE0.9260440900349882
Test R2: 0.2086346072768177, Test RMSE0.8905896046763719
----- Target Encodingを実施したモデル -----
Train R2: 0.37568161006542455, Train RMSE0.822915798523137
Test R2: 0.3541488596070381, Test RMSE0.8045542065519756


In [22]:
h2o.shutdown()

H2O session _sid_bb88 closed.


<ipython-input-22-1edf85295eae>:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


## H2OTargetEncoderEstimatorのblendingに関して
水準内のデータ数に基づいて、目標平均（水準別ターゲット変数の平均）に重みを付ける。 一部の水準のデータ数が少なく、目標平均が信頼できない場合がある。 これを防ぐために、水準における目標平均とグローバル平均の加重平均を取る。

参考：[YouTube - Automatic Feature Engineering with Driverless AI](https://www.youtube.com/watch?v=VMTKcT1iHww)  
17:30辺り

参考によると、ある水準における目標平均は以下：
    
$$
    λ(n~level) * mean(level) + (1-λ(n~level) ) * mean(Global) \\
    λ(n~level) = 1 / 1 + exp(-(n~level - inflection\_point)/smoothing) \\
$$
ある水準のデータ数（$n~level$）が少なければ、目標平均はグローバル平均に近づく